# **Prediksi Curah Hujan 44 Daerah di Singapura pada tahun 2024 - 2025**

Tim: **Lagi KKN**

Anggota:
1. Fitri Hartanti
2. Marta Afifah
3. Hanna Safira Choirunisa

# **Business Understanding**

### **Pendahuluan**

Dalam beberapa tahun terakhir, peningkatan frekuensi dan intensitas peristiwa cuaca ekstrem menjadi isu global yang semakin mendesak. Singapura, sebagai negara kecil dengan kepadatan penduduk tinggi dan infrastruktur perkotaan yang kompleks, turut merasakan dampak perubahan iklim ini. Fenomena seperti banjir lokal akibat hujan deras maupun gelombang panas yang memengaruhi kesehatan masyarakat telah menjadi perhatian serius.

Meteorological Service Singapore (MSS), sebagai lembaga resmi pengamatan dan prakiraan cuaca di Singapura, memiliki 44 stasiun pengamatan yang secara rutin mengumpulkan data harian. Namun, kompleksitas pola cuaca mikro yang sangat bervariasi di tiap wilayah serta keterbatasan data historis dalam format terstruktur menjadikan prediksi cuaca tetap menjadi tantangan besar. Di sisi lain, banyak data historis yang hanya tersedia dalam bentuk visual seperti grafik pada dokumen PDF atau gambar statis, sehingga sulit dimanfaatkan langsung untuk analisis kuantitatif.

Di tengah pesatnya perkembangan teknologi kecerdasan buatan, computer vision, dan machine learning, muncul peluang baru untuk mengekstraksi data dari gambar plot time series cuaca menjadi format numerik yang dapat digunakan untuk pelatihan model prediktif. Pendekatan ini tidak hanya berguna untuk meningkatkan akurasi prakiraan cuaca di Singapura, tetapi juga dapat diadaptasi untuk negara lain seperti Indonesia yang memiliki risiko serupa terhadap bencana cuaca ekstrem.

### **Tujuan**

Proyek ini bertujuan untuk:

1. **Mengekstraksi data numerik dari gambar grafik cuaca** menggunakan teknik *computer vision* dan *OCR* secara otomatis dan akurat.
2. **Membangun model prediksi cuaca** berdasarkan data historis yang diperoleh dari hasil ekstraksi grafik.
3. **Meningkatkan akurasi prediksi cuaca ekstrem** dengan memperluas basis data historis yang dapat dianalisis oleh model.
4. **Mendorong pemanfaatan AI dan machine learning** dalam mendukung pengambilan keputusan yang lebih cepat dan tepat terkait mitigasi bencana iklim.
5. **Mendemonstrasikan relevansi lintas negara**, khususnya bagaimana solusi ini dapat diterapkan di Indonesia untuk memperkuat sistem peringatan dini dan pengelolaan risiko bencana.


### **Fokus Proyek**

Fokus utama proyek ini adalah mengembangkan solusi inovatif berbasis *computer vision* dan *machine learning* untuk memproses dan memprediksi data cuaca dari grafik gambar. Ruang lingkup pekerjaan meliputi:

* Identifikasi dan ekstraksi sumbu, skala, dan titik data dari grafik cuaca (plot time series).
* Transformasi data visual menjadi data numerik terstruktur yang dapat digunakan dalam analisis kuantitatif.
* Pelatihan dan evaluasi model prediksi cuaca menggunakan data historis hasil ekstraksi.
* Evaluasi performa model menggunakan metrik Mean Squared Error (MSE).
* Validasi metode agar dapat direplikasi dan diadaptasi untuk wilayah lain, terutama Indonesia, yang memiliki kebutuhan mendesak terhadap prakiraan cuaca berbasis data.


## Dataset
Kaggle Competition  <br>
Link: https://www.kaggle.com/competitions/sebelas-maret-statistics-data-science-2025/data

Kami menggunakan Webplotdigitizer untuk mengekstrak data gambar plot daily rainfall menjadi data tabular. Berikut adalah data ekstraksi plot daily rainfall yang diperoleh, <br>
Link: https://drive.google.com/drive/folders/1Bswry-FX-nvqz4iwTJxVkglJTi8Z5n8E?usp=sharing

Kami hanya menggunakan data historis harian selama 3 tahun ke belakang yaitu 2021 hingga 2023 karena mempertimbangkan relevansi cuaca. Data ini digunakan untuk melakukan prediksi curah hujan pada januari 2024 - mei 2025.

## Import Library

In [ ]:
# libraries umum yang sering digunakan
import os
import shutil
import random
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd
from functools import reduce

# libraries untuk visualisasi
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# libraries untuk preprocessing
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# libraries untuk pembuatan dan evaluasi model
from sklearn.ensemble import BaggingRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
import joblib

# abaikan peringatan
import warnings
warnings.filterwarnings("ignore")

# set semua seed untuk reproducibility
SEED = 42

random.seed(SEED)
np.random.seed(SEED)

## Data Preparation

In [ ]:
# import kaggle api
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"martaafifah","key":"7851876fbabebdcc3effc753bd26618d"}'}

In [ ]:
# buat folder .kaggle
if not os.path.exists('/root/.kaggle'):
    os.makedirs('/root/.kaggle')

# memindahkan file kaggle.json ke folder contentA
shutil.move('/content/kaggle.json', '/root/.kaggle/kaggle.json')

# mengatur hak akses file untuk keamanan
os.chmod('/root/.kaggle/kaggle.json', 0o600)

In [ ]:
!kaggle competitions download -c sebelas-maret-statistics-data-science-2025

!unzip sebelas-maret-statistics-data-science-2025.zip

 83% 111M/133M [00:00<00:00, 1.16GB/s]
100% 133M/133M [00:00<00:00, 1.04GB/s]
Archive:  sebelas-maret-statistics-data-science-2025.zip
  inflating: Data Eksternal/AirQualityIndex_Google Trends.csv  
  inflating: Data Eksternal/Dipole Mode Index (DMI).csv  
  inflating: Data Eksternal/OceanicNinoIndex (ONI).csv  
  inflating: Data Eksternal/RelativeHumidityMonthlyMean.csv  
  inflating: Test/Admiralty/Data_Gabungan_Lainnya_2024.csv  
  inflating: Test/Admiralty/Data_Gabungan_Lainnya_2025.csv  
  inflating: Test/Ang_Mo_Kio/Data_Gabungan_Lainnya_2024.csv  
  inflating: Test/Ang_Mo_Kio/Data_Gabungan_Lainnya_2025.csv  
  inflating: Test/Bukit_Panjang/Data_Gabungan_Lainnya_2024.csv  
  inflating: Test/Bukit_Panjang/Data_Gabungan_Lainnya_2025.csv  
  inflating: Test/Bukit_Timah/Data_Gabungan_Lainnya_2024.csv  
  inflating: Test/Bukit_Timah/Data_Gabungan_Lainnya_2025.csv  
  inflating: Test/Buona_Vista/Data_Gabungan_Lainnya_2024.csv  
  inflating: Test/Buona_Vista/Data_Gabungan_Lainnya_2025.cs

In [ ]:
# mount gdrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
locations_1 = [
    'Admiralty', 'Ang_Mo_Kio', 'Bukit_Panjang', 'Bukit_Timah', 'Buona_Vista', 'Changi',
    'Choa_Chu_Kang_Central', 'Choa_Chu_Kang_South', 'Clementi', 'East_Coast_Parkway',
    'Jurong_Island', 'Jurong_Pier', 'Jurong_West', 'Kent_Ridge', 'Kranji_Reservoir'
]

In [ ]:
locations_2 = [
    'Lim_Chu_Kang', 'Lower_Peirce_Reservoir', 'Macritchie_Reservoir', 'Mandai',
    'Marina_Barrage', 'Marine_Parade', 'Newton', 'Nicoll_Highway', 'Pasir_Panjang',
    'Pasir_Ris_Central', 'Pasir_Ris_West', 'Paya_Lebar', 'Pulau_Ubin', 'Punggol', 'Queenstown',
    'Seletar', 'Sembawang', 'Sentosa_Island', 'Simei', 'Somerset_Road', 'Tai_Seng',
    'Tanjong_Katong', 'Tengah', 'Toa_Payoh', 'Tuas', 'Tuas_South',
    'Ulu_Pandan', 'Upper_Peirce_Reservoir', 'Whampoa'
]

In [ ]:
# daftar nama wilayah
locations = locations_1 + locations_2

### Data Train & Test

In [ ]:
# Inisialisasi dictionary
X_train = {}
X_test = {}
y_train = {}

In [ ]:
# import data fitur train dan test
for loc in locations:
    # data fitur train
    X_train[loc] = pd.concat([
        pd.read_csv(f'/content/Train/{loc}/Data_Gabungan_Lainnya_2021.csv'),
        pd.read_csv(f'/content/Train/{loc}/Data_Gabungan_Lainnya_2022.csv'),
        pd.read_csv(f'/content/Train/{loc}/Data_Gabungan_Lainnya_2023.csv')
    ], ignore_index=True)

    # data fitur test
    X_test[loc] = pd.concat([
        pd.read_csv(f'/content/Test/{loc}/Data_Gabungan_Lainnya_2024.csv'),
        pd.read_csv(f'/content/Test/{loc}/Data_Gabungan_Lainnya_2025.csv')
    ], ignore_index=True)

In [ ]:
# import data target train

# locations 1
for loc in locations_1:
    # data target train
    y_train[loc] = pd.concat([
        pd.read_csv(f'/content/drive/MyDrive/Daily Rainfall Total (mm)/{loc}/rainfall_{loc.lower()}_2021.csv'),
        pd.read_csv(f'/content/drive/MyDrive/Daily Rainfall Total (mm)/{loc}/rainfall_{loc.lower()}_2022.csv'),
        pd.read_csv(f'/content/drive/MyDrive/Daily Rainfall Total (mm)/{loc}/rainfall_{loc.lower()}_2023.csv')
    ], ignore_index=True)

# locations 2
for loc in locations_2:
    # data target train
    y_train[loc] = pd.concat([
        pd.read_csv(f'/content/drive/MyDrive/Daily Rainfall Total (mm)/{loc}/rainfall_{loc.lower()}_2021.csv', delimiter=';'),
        pd.read_csv(f'/content/drive/MyDrive/Daily Rainfall Total (mm)/{loc}/rainfall_{loc.lower()}_2022.csv', delimiter=';'),
        pd.read_csv(f'/content/drive/MyDrive/Daily Rainfall Total (mm)/{loc}/rainfall_{loc.lower()}_2023.csv', delimiter=';')
    ], ignore_index=True)

### Data Eksternal

In [ ]:
aqi = pd.read_csv('/content/Data Eksternal/AirQualityIndex_Google Trends.csv')
dmi = pd.read_csv('/content/Data Eksternal/Dipole Mode Index (DMI).csv')
oni = pd.read_csv('/content/Data Eksternal/OceanicNinoIndex (ONI).csv')
humidity = pd.read_csv('/content/Data Eksternal/RelativeHumidityMonthlyMean.csv')

## **Preprocessing**
Sebelum melakukan analisis lebih lanjut, dilakukan preprocessing data terlenih dahulu untuk memperoleh data yang valid dan siap untuk dianalisis.

#### Data Eksternal
Digunakan data eksternal yang disediakan oleh panitia, yaitu AQI, DMI, ONI, dan Humidity. Data ini dipilih karena dianggap masih relevan untuk mendukung peramalan cuaca. Keempat variabel tersebut tersedia dalam format bulanan, sementara target prediksi berupa curah hujan harian (daily rainfall). Oleh karena itu, data eksternal diubah menjadi format harian dengan cara menggandakan nilai bulanannya sehingga setiap hari dalam bulan yang sama memiliki nilai AQI, DMI, ONI, dan humidity yang identik. Proses  preprocessing untuk masing-masing variabel dijelaskan sebagai berikut:

##### AQI

In [ ]:
aqi.head(3)

,Bulan,air quality index: (Singapura)
0,2004-01,0
1,2004-02,0
2,2004-03,0


In [ ]:
# ubah tipe kolom Bulan menjadi datetime
aqi['Bulan'] = pd.to_datetime(aqi['Bulan'])

# rename kolom nilai AQI
aqi.rename(columns={'air quality index: (Singapura)': 'AQI'}, inplace=True)

In [ ]:
# filter dari Januari 2021 sampai Mei 2025
aqi = aqi[(aqi['Bulan'] >= '2021-01-01') & (aqi['Bulan'] <= '2025-05-31')]

In [ ]:
def monthly_to_daily(df, value_col, date_col='Date'):
    daily_data = []
    for _, row in df.iterrows():
        start_date = row[date_col]
        end_date = start_date + pd.offsets.MonthEnd(0)
        date_range = pd.date_range(start=start_date, end=end_date, freq='D')
        for date in date_range:
            daily_data.append({'Date': date, value_col: row[value_col]})
    return pd.DataFrame(daily_data)

In [ ]:
daily_aqi = monthly_to_daily(aqi, 'AQI', 'Bulan')

In [ ]:
daily_aqi.head(3)

,Date,AQI
0,2021-01-01,0
1,2021-01-02,0
2,2021-01-03,0


##### DMI

In [ ]:
dmi.head(3)

,Date,DMI HadISST1.1 missing value -9999 https://psl.noaa.gov/data/timeseries/month/
0,1870-01-01,-0.438
1,1870-02-01,-0.336
2,1870-03-01,0.177


In [ ]:
# ubah tipe kolom Date ke datetime
dmi['Date'] = pd.to_datetime(dmi['Date'])

# rename kolom nilai DMI
dmi.rename(columns={' DMI HadISST1.1  missing value -9999 https://psl.noaa.gov/data/timeseries/month/': 'DMI'}, inplace=True)

In [ ]:
# filter dari Januari 2021 sampai Mei 2025
dmi = dmi[(dmi['Date'] >= '2021-01-01') & (dmi['Date'] <= '2025-05-31')]

In [ ]:
# ganti -9999 dengan NaN
dmi['DMI'] = dmi['DMI'].replace(-9999, pd.NA)

# isi missing value dengan nilai sebelumnya
dmi['DMI'] = dmi['DMI'].fillna(method='ffill')

In [ ]:
dmi.isna().sum()

,0
Date,0
DMI,0


In [ ]:
daily_dmi = monthly_to_daily(dmi, 'DMI')

In [ ]:
daily_dmi.head(3)

,Date,DMI
0,2021-01-01,0.051
1,2021-01-02,0.051
2,2021-01-03,0.051


##### ONI

In [ ]:
oni.head(3)

,Date,ONI
0,01/01/1950,-1.53
1,01/02/1950,-1.34
2,01/03/1950,-1.16


In [ ]:
# ubah kolom Date ke datetime
oni['Date'] = pd.to_datetime(oni['Date'], format='%d/%m/%Y')

# rename kolom nilai ONI
oni.rename(columns={'  ONI': 'ONI'}, inplace=True)

In [ ]:
# filter dari Januari 2021 sampai Mei 2025
oni = oni[(oni['Date'] >= '2021-01-01') & (oni['Date'] <= '2025-05-31')]

In [ ]:
# ganti -9999 dengan NaN
oni['ONI'] = oni['ONI'].replace(-9999, pd.NA)

# isi missing value dengan nilai sebelumnya
oni['ONI'] = oni['ONI'].fillna(method='ffill')

In [ ]:
oni.isna().sum()

,0
Date,0
ONI,0


In [ ]:
daily_oni = monthly_to_daily(oni, 'ONI')

In [ ]:
daily_oni.head(3)

,Date,ONI
0,2021-01-01,-1.05
1,2021-01-02,-1.05
2,2021-01-03,-1.05


##### Humidity

In [ ]:
humidity.tail(3)

,month,mean_rh
518,2025-03,82.4
519,2025-04,81.3
520,2025-05,79.2


Datanya cuma sampai mei

In [ ]:
# ubah tipe kolom Date ke datetime
humidity['month'] = pd.to_datetime(humidity['month'])

# rename kolom nilai humidity
humidity.rename(columns={'mean_rh': 'humidity'}, inplace=True)

In [ ]:
# filter hanya tahun 2021–2025
humidity = humidity[(humidity['month'].dt.year >= 2021) & (humidity['month'].dt.year <= 2025)]

In [ ]:
daily_humidity = monthly_to_daily(humidity, 'humidity', 'month')

In [ ]:
daily_humidity.head(3)

,Date,humidity
0,2021-01-01,82.6
1,2021-01-02,82.6
2,2021-01-03,82.6


##### Gabungkan keempat data eksternal


In [ ]:
external_list = [daily_aqi, daily_dmi, daily_oni, daily_humidity]
external = reduce(lambda left, right: pd.merge(left, right, on='Date', how='outer'), external_list)

In [ ]:
# bagi data eksternal gabungan menjadi train dan test
external_train = external[(external['Date'] >= '2021-01-01') & (external['Date'] <= '2023-12-31')].copy()
external_test  = external[(external['Date'] >= '2024-01-01') & (external['Date'] <= '2025-05-31')].copy()

# cek jumlah data
print(f"Train: {len(external_train)} baris")
print(f"Test : {len(external_test)} baris")

Train: 1095 baris
Test : 517 baris


Dari data eksternal yang sudah digabungkan, dilakukan pemisahan data (splitting) menjadi data latih (train) dan data uji (test). Karena data yang digunakan merupakan data runtun waktu (time series), proses pemisahan dilakukan dengan mempertimbangkan urutan kronologis berdasarkan tahun. Data latih mencakup periode tahun 2021 hingga 2023, dengan total 1095 baris data. Sementara itu, data uji mencakup tahun 2024 hingga 2025, dengan jumlah 517 baris data.

In [ ]:
# ubah tipe kolom Date ke datetime
external_train['Date'] = pd.to_datetime(external_train['Date'])
external_test['Date'] = pd.to_datetime(external_test['Date'])

### Data Train & Test

#### Perbaikan data train Nicoll Highway 2022

In [ ]:
df_nicoll = X_train['Nicoll_Highway']

In [ ]:
# Buat range semua tanggal ideal di 2023
all_dates = pd.date_range(start='2021-01-01', end='2023-12-31')

# Cek tanggal yang hilang
existing_dates = pd.to_datetime(df_nicoll['Date'])
missing_dates = all_dates.difference(existing_dates)

print("Jumlah tanggal hilang:", len(missing_dates))
print(missing_dates)

Jumlah tanggal hilang: 30
DatetimeIndex(['2023-11-01', '2023-11-02', '2023-11-03', '2023-11-04',
               '2023-11-05', '2023-11-06', '2023-11-07', '2023-11-08',
               '2023-11-09', '2023-11-10', '2023-11-11', '2023-11-12',
               '2023-11-13', '2023-11-14', '2023-11-15', '2023-11-16',
               '2023-11-17', '2023-11-18', '2023-11-19', '2023-11-20',
               '2023-11-21', '2023-11-22', '2023-11-23', '2023-11-24',
               '2023-11-25', '2023-11-26', '2023-11-27', '2023-11-28',
               '2023-11-29', '2023-11-30'],
              dtype='datetime64[ns]', freq='D')


Dari hasil pengecekan ditemukan bahwa data bulan November 2023 tidak ditemukan.

In [ ]:
# buat DataFrame baru berisi semua tanggal
df_filled = pd.DataFrame({'Date': all_dates})

# gabungkan dengan data yang tersedia
df_nicoll['Date'] = pd.to_datetime(df_nicoll['Date'])
df_merged = pd.merge(df_filled, df_nicoll, on='Date', how='left')

# isi missing value dengan nilai sebelumnya
df_merged = df_merged.fillna(method='ffill')

In [ ]:
X_train['Nicoll_Highway'] = df_merged

Selanjutnya, dibuat suatu fungsi untuk preprocessing data, dengan langkah sebagai berikut:
1. Mengambil data fitur (X) dan target (y) dari dictionary berdasarkan nama lokasi yang diberikan.
2. Kolom Date pada kedua data tersebut dikonversi ke format datetime agar dapat diproses secara konsisten.
3. Untuk menjaga validitas data, nilai negatif pada kolom Rainfall diubah menjadi nol karena curah hujan tidak dapat bernilai negatif secara logis
4. Data eksternal yang berisi informasi tambahan seperti AQI, DMI, ONI, dan humidity digabungkan ke dalam data fitur berdasarkan tanggal dengan metode left join.
5. Setelah penggabungan, dilakukan pengisian missing value menggunakan metode forward fill (ffill) dan backward fill (bfill) secara berurutan. Metode ini membantu mengisi nilai kosong dengan data sebelumnya atau sesudahnya untuk menjaga kontinuitas data, baik pada fitur maupun target
6. Dilakukan ekstraksi fitur waktu dari kolom Date, yaitu tahun, bulan, dan hari, untuk menambahkan informasi musiman yang penting dalam data cuaca. Kemudian, kolom Date diatur sebagai indeks pada kedua data, agar urutan kronologis tetap terjaga dan mendukung struktur data time series.

Langkah serupa juga diterapkan pada data uji. Data uji terlebih dahulu dikonversi format tanggalnya, kemudian digabungkan dengan data eksternal uji, dan missing value diisi dengan metode yang sama. Setelah itu, dilakukan ekstraksi fitur waktu dan penetapan kolom Date sebagai indeks. Dengan cara ini, baik data latih maupun data uji memiliki format dan struktur yang sama serta siap digunakan dalam proses pelatihan dan evaluasi model prediktif.

In [ ]:
# fungsi preprocessing data per lokasi
def preprocess_location(loc_name, X_train_dict, y_train_dict, X_test_dict, external_train, external_test):
    # ---- Train ----
    X = X_train_dict[loc_name]
    y = y_train_dict[loc_name]

    # pastikan tanggal dalam format datetime
    X['Date'] = pd.to_datetime(X['Date'])
    y['Date'] = pd.to_datetime(y['Date'])

    # ganti nilai negatif pada target menjadi 0
    y['Rainfall'] = y['Rainfall'].clip(lower=0)

    # gabung data eksternal berdasarkan tanggal
    X = pd.merge(X, external_train, on='Date', how='left')

    # isi missing value pada data fitur dengan metode ffill dan bfill
    X.fillna(method='ffill', inplace=True)
    X.fillna(method='bfill', inplace=True)

    # isi missing value pada data target
    y.fillna(method='ffill', inplace=True)
    y.fillna(method='bfill', inplace=True)

    # ekstraksi fitur waktu
    X['year'] = X['Date'].dt.year
    X['month'] = X['Date'].dt.month
    X['day'] = X['Date'].dt.day

    # set kolom Date sebagai index
    X = X.set_index('Date')
    y = y.set_index('Date')

    # ---- Test ----
    X_test = X_test_dict[loc_name]
    X_test['Date'] = pd.to_datetime(X_test['Date'])

    # gabung data eksternal
    X_test = pd.merge(X_test, external_test, on='Date', how='left')

    # isi missing value
    X_test.fillna(method='ffill', inplace=True)
    X_test.fillna(method='bfill', inplace=True)

    # ekstraksi fitur waktu
    X_test['year'] = X_test['Date'].dt.year
    X_test['month'] = X_test['Date'].dt.month
    X_test['day'] = X_test['Date'].dt.day

    # set kolom Date sebagai index
    X_test = X_test.set_index('Date')

    return X, y, X_test

In [ ]:
# dictionary untuk menyimpan hasil
X_train_clean = {}
y_train_clean = {}
X_test_clean = {}

for loc in locations:
    X_, y_, Xtest_ = preprocess_location(loc, X_train, y_train, X_test, external_train, external_test)
    X_train_clean[loc] = X_
    y_train_clean[loc] = y_
    X_test_clean[loc] = Xtest_
    y_train_clean[loc].index = X_train_clean[loc].index

#### Clustering

In [ ]:
parameter_cuaca = ['Highest 30 min Rainfall (mm)', 'Highest 60 min Rainfall (mm)',
                'Highest 120 min Rainfall (mm)', 'Mean Temperature (°C)',
                'Maximum Temperature (°C)', 'Minimum Temperature (°C)',
                'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)']

In [ ]:
# agregat per lokasi
agg_data = []

for loc in locations:
    df = X_train_clean[loc][parameter_cuaca]
    agg_row = df.agg(['mean', 'std']).T
    agg_row.columns = ['mean', 'std']
    agg_row['loc'] = loc
    agg_data.append(agg_row)

agg_df = pd.concat(agg_data, keys=[loc for loc in locations])
agg_df = agg_df.reset_index()
agg_df = agg_df.pivot(index='level_0', columns='level_1', values=['mean', 'std'])
agg_df.columns = ['_'.join(col).strip() for col in agg_df.columns.values]
agg_df.index.name = 'loc'
agg_df.reset_index(inplace=True)

In [ ]:
agg_df.head(3)

,loc,mean_Highest 120 min Rainfall (mm),mean_Highest 30 min Rainfall (mm),mean_Highest 60 min Rainfall (mm),mean_Max Wind Speed (km/h),mean_Maximum Temperature (°C),mean_Mean Temperature (°C),mean_Mean Wind Speed (km/h),mean_Minimum Temperature (°C),std_Highest 120 min Rainfall (mm),std_Highest 30 min Rainfall (mm),std_Highest 60 min Rainfall (mm),std_Max Wind Speed (km/h),std_Maximum Temperature (°C),std_Mean Temperature (°C),std_Mean Wind Speed (km/h),std_Minimum Temperature (°C)
0,Admiralty,6.490411,4.438904,5.608311,37.016986,31.861826,27.571507,10.649589,24.848128,12.780727,8.179223,10.808601,9.531784,1.984507,1.274268,5.114191,1.300033
1,Ang_Mo_Kio,7.056073,4.889680,6.118721,30.495434,32.024658,27.904475,5.787123,25.116895,13.200624,8.729593,11.268830,6.457040,1.933954,1.343162,1.546308,1.425857
2,Bukit_Panjang,7.495799,5.123927,6.451689,NaN,NaN,NaN,NaN,NaN,14.059020,8.753948,11.611979,NaN,NaN,NaN,NaN,NaN


In [ ]:
# ganti missing value pada agg_df dengan median per kolom
agg_df_imputed = agg_df.drop(columns='loc').copy()
agg_df_imputed = agg_df_imputed.fillna(agg_df_imputed.median())

In [ ]:
# standardisasi
scl = StandardScaler()
features_scaled = scl.fit_transform(agg_df_imputed)

# KMeans clustering
kmeans = KMeans(n_clusters=5, random_state=SEED)
agg_df['cluster'] = kmeans.fit_predict(features_scaled)

# buat mapping lokasi ke cluster
loc_to_cluster = dict(zip(agg_df['loc'], agg_df['cluster']))

#### Lokasi di gabungkan

In [ ]:
# gabungkan semua lokasi
X_train_all = pd.concat([X_train_clean[loc] for loc in locations], axis=0)
y_train_all = pd.concat([
    pd.Series(y_train_clean[loc].iloc[:, 0].values, index=X_train_clean[loc].index)  # ambil hanya hari ke-1
    for loc in locations
], axis=0)
X_test_all = pd.concat([X_test_clean[loc] for loc in locations], axis=0)

# pastikan indeksnya sejajar
y_train_all.index = X_train_all.index

In [ ]:
# tambahkan fitur lokasi sebagai variabel kategorik
def get_location_feature(df, loc_name):
    df = df.copy()
    df['lokasi'] = loc_name
    return df

In [ ]:
X_train_all = pd.concat([
    get_location_feature(X_train_clean[loc], loc) for loc in locations
], axis=0)

X_test_all = pd.concat([
    get_location_feature(X_test_clean[loc], loc) for loc in locations
], axis=0)

In [ ]:
# tambahkan kolom 'cluster' berdasarkan lokasi
X_train_all['cluster'] = X_train_all['lokasi'].map(
    lambda x: loc_to_cluster[x]
)
X_test_all['cluster'] = X_test_all['lokasi'].map(
    lambda x: loc_to_cluster[x]
)

In [ ]:
numeric_cols = ['Highest 30 min Rainfall (mm)', 'Highest 60 min Rainfall (mm)',
                'Highest 120 min Rainfall (mm)', 'Mean Temperature (°C)',
                'Maximum Temperature (°C)', 'Minimum Temperature (°C)',
                'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)',
                'AQI', 'DMI', 'ONI', 'humidity']

In [ ]:
# isi missing value dengan median per hari per cluster
def fillna_daily_cluster_median(df, numeric_cols):

    # hitung median harian per cluster
    daily_cluster_medians = df.groupby(['cluster', 'month', 'day'])[numeric_cols].median()

    # imputasi missing value menggunakan median dari cluster di tanggal yang sama
    for col in numeric_cols:
        df[col] = df.apply(
            lambda row: daily_cluster_medians.loc[(row['cluster'], row['month'], row['day']), col]
            if pd.isna(row[col]) else row[col],
            axis=1
        )

    return df

In [ ]:
X_train_all = fillna_daily_cluster_median(X_train_all, numeric_cols)
X_test_all = fillna_daily_cluster_median(X_test_all, numeric_cols)

In [ ]:
# standarisasi
X_train_all[numeric_cols] = scl.fit_transform(X_train_all[numeric_cols])
X_test_all[numeric_cols] = scl.transform(X_test_all[numeric_cols])

In [ ]:
# encode variabel lokasi
le = LabelEncoder()
X_train_all['lokasi'] = le.fit_transform(X_train_all['lokasi'])
X_test_all['lokasi'] = le.transform(X_test_all['lokasi'])

## Pemodelan

Model yang digunakan adalah **ExtraTreesRegressor**, pemilihan model ini sebagai model prediksi dalam proyek  didasarkan pada beberapa pertimbangan teknis dan karakteristik data cuaca historis yang kompleks serta berdimensi tinggi. Berikut adalah alasan utama penggunaannya:

1. **Kemampuan Menangani Data Non-Linear dan Kompleks**
   Data cuaca umumnya bersifat non-linear dan dipengaruhi oleh interaksi banyak variabel (seperti lokasi, hari, bulan, dan tahun). ExtraTreesRegressor, sebagai metode ansambel berbasis pohon keputusan, mampu menangkap pola non-linear dan kompleks tanpa memerlukan transformasi data secara eksplisit.

2. **Model Ansambel yang Stabil dan Tidak Overfitting**
   Dibandingkan dengan pohon keputusan tunggal, ExtraTrees menggunakan banyak pohon dengan pemilihan split yang dilakukan secara acak (*extremely randomized trees*), yang cenderung mengurangi *overfitting* serta meningkatkan generalisasi model terhadap data yang belum pernah dilihat.

3. **Kinerja Cepat dan Efisien**
   ExtraTrees cenderung lebih cepat dibandingkan Random Forest karena pemilihan split dilakukan secara acak, bukan berdasarkan pencarian optimal. Ini memberikan keunggulan efisiensi dalam eksperimen dan prediksi cepat dalam proyek berskala besar.

4. **Tanpa Perlu Normalisasi atau Skaling Data**
   Karena model ini berbasis pada *decision trees*, tidak diperlukan proses normalisasi atau skaling pada fitur numerik, sehingga mempercepat pipeline data dan memudahkan implementasi.

5. **Akurasi yang Kompetitif dengan Interpretabilitas yang Cukup Baik**
   Dengan fitur-fitur penting yang bisa diekstrak dari model, ExtraTrees memungkinkan analisis kontribusi fitur terhadap prediksi (misalnya melalui `feature_importances_`), yang bermanfaat dalam interpretasi model.

6. **Cocok untuk Dataset Historis Berbasis Waktu dan Lokasi**
   Dataset ini melibatkan banyak kombinasi spasial dan temporal (lokasi dan waktu), sehingga model yang dapat menangani interaksi variabel secara fleksibel seperti ExtraTrees sangat relevan digunakan.

In [ ]:
X_train_all

,Highest 30 min Rainfall (mm),Highest 60 min Rainfall (mm),Highest 120 min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),AQI,DMI,ONI,humidity,year,month,day,lokasi,cluster
Date,,,,,,,,,,,,,,,,,
2021-01-01,1.616291,1.696737,1.974946,-3.476757,-4.214069,-3.291812,2.277648,2.587929,-0.531086,-0.212649,-0.813043,1.800706,2021,1,1,0,0
2021-01-02,1.422042,1.641236,2.037548,-4.550201,-5.175135,-3.291812,1.385626,1.996723,-0.531086,-0.212649,-0.813043,1.800706,2021,1,2,0,0
2021-01-03,-0.423326,-0.393785,-0.388281,-3.745118,-4.282717,-1.994122,1.625786,-0.777396,-0.531086,-0.212649,-0.813043,1.800706,2021,1,3,0,0
2021-01-04,-0.520451,-0.504786,-0.497835,-2.582222,-2.909766,-2.167147,2.414882,-0.307463,-0.531086,-0.212649,-0.813043,1.800706,2021,1,4,0,0
2021-01-05,-0.544732,-0.523286,-0.513485,-1.061510,-0.781691,-1.215508,3.512756,1.557109,-0.531086,-0.212649,-0.813043,1.800706,2021,1,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,-0.107671,-0.171783,-0.137873,-0.480062,0.179375,0.082183,0.201981,0.018458,-0.531086,1.788449,2.354621,1.840382,2023,12,27,43,1
2023-12-28,2.223320,1.863238,1.974946,-0.524789,-0.335482,0.082183,0.030439,0.154890,-0.531086,1.788449,2.354621,1.840382,2023,12,28,43,1
2023-12-29,-0.399045,-0.356785,-0.310028,-0.032794,-0.026568,0.082183,-0.226875,-0.201349,-0.531086,1.788449,2.354621,1.840382,2023,12,29,43,1


In [ ]:
# fit model extra trees pada semua data
extratrees = ExtraTreesRegressor(n_estimators=100, random_state=SEED)
extratrees.fit(X_train_all, y_train_all)

ExtraTreesRegressor(random_state=42)

In [ ]:
# prediksi pada data training
y_train_pred = extratrees.predict(X_train_all)

# hitung MSE
mse_train = mean_squared_error(y_train_all, y_train_pred)

print(f"MSE (Train): {mse_train}")

MSE (Train): 5.5008790918669605e-21


Diperoleh nilai MSE untuk data train sebesar 5,6739467934605824e-21. Nilai MSE yang mendekati nol ini menunjukkan bahwa model memiliki tingkat kesalahan prediksi yang sangat kecil terhadap data yang digunakan dalam pelatihan. Dengan kata lain, prediksi model terhadap data latih sangat mendekati nilai aktual, yang mengindikasikan tingkat akurasi yang sangat tinggi selama proses pelatihan. Selanjutnya, dilakukan prediksi nilai Rainfall pada data test menggunakan model yang sudah diperoleh, sebagai berikut:

In [ ]:
X_test_all

,Highest 30 min Rainfall (mm),Highest 60 min Rainfall (mm),Highest 120 min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),AQI,DMI,ONI,humidity,year,month,day,lokasi,cluster
Date,,,,,,,,,,,,,,,,,
2024-01-01,-0.301921,-0.338285,-0.278727,-2.045500,-2.429233,-0.350381,-0.398418,-0.019440,-0.531086,1.573331,2.175120,2.237141,2024,1,1,0,0
2024-01-02,-0.484029,-0.477036,-0.396106,-0.703696,0.179375,-0.609919,0.356370,0.617243,-0.531086,1.573331,2.175120,2.237141,2024,1,2,0,0
2024-01-03,-0.362624,-0.384535,-0.396106,-0.524789,-0.713043,0.082183,0.939615,1.299404,-0.531086,1.573331,2.175120,2.237141,2024,1,3,0,0
2024-01-04,0.742169,0.753227,1.223722,-2.761129,-3.664889,-1.648071,0.596530,-0.610646,-0.531086,1.573331,2.175120,2.237141,2024,1,4,0,0
2024-01-05,-0.277640,-0.227283,-0.231776,-1.687686,-1.742757,-0.869457,1.454243,1.011380,-0.531086,1.573331,2.175120,2.237141,2024,1,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-27,-0.544732,-0.523286,-0.513485,1.890458,1.174765,2.634306,-0.295492,-0.921408,0.065824,0.032486,0.200609,0.451726,2025,5,27,43,1
2025-05-28,-0.544732,-0.523286,-0.513485,1.801005,1.106117,1.898949,-0.226875,-0.428736,0.065824,0.032486,0.200609,0.451726,2025,5,28,43,1
2025-05-29,-0.350483,-0.264284,-0.200475,2.024639,1.346384,2.115230,-0.141104,0.374697,0.065824,0.032486,0.200609,0.451726,2025,5,29,43,1


In [ ]:
y_pred_all = extratrees.predict(X_test_all)

In [ ]:
# salin kembali dataframe
result_df = X_test_all.copy()
result_df['prediksi'] = y_pred_all

# decode label lokasi
result_df['lokasi'] = le.inverse_transform(result_df['lokasi'])

# reset index
result_df = result_df.reset_index(drop=True)

# buat kolom ID (kota)
result_df['ID (kota)'] = result_df.apply(
    lambda row: f"{row['lokasi'].replace('_', ' ').title().replace(' ', '_')}_{int(row['year']):04d}_{int(row['month']):02d}_{int(row['day']):02d}",
    axis=1
)

result_df['ID (kota)'] = result_df['ID (kota)'].str.lower()

# susun kolom akhir
result_df = result_df[['ID (kota)', 'year', 'month', 'day', 'prediksi']]
result_df.columns = ['ID (kota)', 'tahun', 'bulan', 'hari', 'prediksi']

In [ ]:
result_df

,ID (kota),tahun,bulan,hari,prediksi
0,admiralty_2024_01_01,2024,1,1,7.367679
1,admiralty_2024_01_02,2024,1,2,3.520632
2,admiralty_2024_01_03,2024,1,3,6.796330
3,admiralty_2024_01_04,2024,1,4,37.550752
4,admiralty_2024_01_05,2024,1,5,7.609381
...,...,...,...,...,...
22743,whampoa_2025_05_27,2025,5,27,1.377189
22744,whampoa_2025_05_28,2025,5,28,1.548236
22745,whampoa_2025_05_29,2025,5,29,7.229875
22746,whampoa_2025_05_30,2025,5,30,1.108796


## Cek apakah data prediksi sesuai format submission

In [ ]:
# Load data sample submission
sample_submission = pd.read_csv('sample_submission_final.csv')

# Cek apakah kolom 'ID (kota)' identik (urutan dan isi)
identik = result_df['ID (kota)'].equals(sample_submission['ID (kota)'])

# Cek apakah isi kolom 'ID (kota)' sama tapi urutannya mungkin beda
sama_tapi_tidak_berurutan = set(result_df['ID (kota)']) == set(sample_submission['ID (kota)'])

# Tampilkan hasil
print("Apakah ID (kota) identik (urutan & isi sama)?", identik)
print("Apakah ID (kota) sama (tanpa memperhatikan urutan)?", sama_tapi_tidak_berurutan)

Apakah ID (kota) identik (urutan & isi sama)? True
Apakah ID (kota) sama (tanpa memperhatikan urutan)? True


## Simpan file prediksi dan model

In [ ]:
result_df.to_csv("Prediksi_Lagi_KKN_8.csv", index=False)

In [ ]:
# simpan model
joblib.dump(extratrees, 'Model_Lagi_KKN_8.pkl')

['Model_Lagi_KKN_8.pkl']

In [ ]:
from google.colab import files

data_prediksi = '/content/Prediksi_Lagi_KKN_8.csv'
model = '/content/Model_Lagi_KKN_8.pkl'

files.download(data_prediksi)
files.download(model)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>